Este agarra los audios en wav/, los procesa, y carga los spectrograms finales a spect/

## esto es ya processing, hay que ver donde meterlo

In [ ]:
import os, sys
import pandas as pd
import librosa as lbrs
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import noisereduce as nr

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Para no tener problemas de relative imports with packages
from utils import util

In [ ]:
# Functions
def load_spectrograms(train_data, source, dest):
    ''' Searches in the source dir, the waveform audio files present in the dataframe 
    Then creates the noise-reduced spectrograms and saves them in the dest dir as pngs. '''

    for row in train_data.itertuples(index=False):
        name = row.filename.replace('.ogg', '')
        class_id = row.class_id
        audio_path = os.path.join(source, str(class_id),f"{name}.wav")
        
        wav, srate = lbrs.load(audio_path, sr=32000)
        wav=nr.reduce_noise(y=wav, sr=srate, stationary=False)
        if np.any(~np.isfinite(wav)):
            print(f"Warning: {name} from {class_id} contains nan or inf values, replacing with zeros.")
            # TODO CHECK THESE LINES, CHECK HOW TO PROPERLY HANDLE THIS
            wav = np.nan_to_num(wav, nan=0.0, posinf=0.0, neginf=0.0) # clean up nans or infs
        
        ffourier = lbrs.stft(wav)
        S=lbrs.amplitude_to_db(np.abs(ffourier), ref=np.max)

        class_dir = os.path.join(dest, str(class_id))                             # crear directorio para la clase
        os.makedirs(class_dir, exist_ok=True)
        filename = f"{name}.png"
        path = os.path.join(class_dir, filename)
        plt.imsave(path, S, cmap='inferno', format='png') # save spectrogram as png

In [ ]:
# Define Paths
wavs_dir = os.path.join('..', 'database', 'assets', 'wav')
spect_dir = os.path.join('..', 'database', 'assets', 'spect')
train_data = pd.read_csv(os.path.join('..', 'database', 'meta', 'train_data.csv'))

In [ ]:
# Reset, then load the raw audio files
print("Resetting spectrogram imgs...")
util.clean_dir(spect_dir)

print("Loading spectrogram imgs...")
load_spectrograms(train_data, wavs_dir, spect_dir)